# Applying Pretrained Models

This notebook is about not reinventing the wheel. Training a deep learning model can be expensive - neural networks are hungry for data, computing resources, and engineering time. Fortunately others have invested into building proven models and are providing them for free. In the following we will go through some examples of pretrained models and learn how to apply them for _inference_ without retraining.

## Preamble

In [ ]:
from tensorflow import keras
import numpy
import matplotlib.pyplot as plt
import seaborn
import pandas

## Inception Network for Image Classification

[Inception V3](https://keras.io/applications/#inceptionv3) is a deep convolutional neural network architecture for image classification. A model trained on 1000 classes from the [ImageNet](ImageNet) benchmark data set is provided with `keras`.

The inception archictecure has been designed to solve the following issues:

- The relevant object in the image can have a large variety of positions and sizes. This makes choosing the right kernel size for the convolution operation difficult. 
- Very deep networks are prone to overfitting, which produces models that do not generalize well, and the vanishing gradient problem, which makes training difficult.
- Naively stacking large convolution operations is computationally expensive.

The solution proposed with this architecture: Make the network not just deep but wide - with convolutional filters of different sizes operating on the image at the same time.

The `keras` library comes with a set of pretrained models, including Inception V3:

In [ ]:
from tensorflow.keras.applications import inception_v3

In [ ]:
inception_model = inception_v3.InceptionV3()

The network expects input images to be of a fixed size:

In [ ]:
inception_dim = (299, 299)

This model should be able to classify a large number of everyday objects in images. Let's try a classic example:

In [ ]:
img_url = "https://i.ytimg.com/vi/UwtTSqTbWzg/maxresdefault.jpg"

The following library calls load the data from the URL into an image object:

In [ ]:
import PIL
import requests
import io

In [ ]:
response = requests.get(img_url)
img = PIL.Image.open(io.BytesIO(response.content))

In [ ]:
img

Now resize it to the required input size:

In [ ]:
img = img.resize(inception_dim)
img

We now use `keras` preprocessing tools to convert the image object into a numpy array, having 3 channels for RGB...

In [ ]:
img_a = keras.preprocessing.image.img_to_array(img)
img_a.shape

In [ ]:
seaborn.distplot(img_a.flatten())

... and preprocess the pixel values for the inception model:

In [ ]:
img_a = inception_v3.preprocess_input(img_a)
img_a.shape

In [ ]:
seaborn.distplot(img_a.flatten())

In [ ]:
plt.imshow(img_a)

Now the image is ready to be fed to the model for inference:

In [ ]:
predictions = inception_model.predict(
    numpy.array([img_a])
)


The prediction comes in the form of a vector with a value for each of the 1000 labels in the training set:

In [ ]:
predictions.shape

This is how we get text labels for the highest values:

In [ ]:
predictions_decoded = keras.applications.imagenet_utils.decode_predictions(predictions)[0]

In [ ]:
pandas.DataFrame(
    predictions_decoded,
    columns=["category", "label", "value"]
).set_index("label").plot(kind="bar", ylim=(0,1))


In case you wonder:
    
> A tabby is any domestic cat (Felis catus) with an 'M' on its forehead, stripes by its eyes and across its cheeks, along its back, and around its legs and tail, and, differing by tabby type, characteristic striped, dotted, lined, flecked, banded or swirled patterns on the body—neck, shoulders, sides, flanks, chest and tummy.

### Exercise: Web Inception

**Wrap the code above in a method that takes the URL of an image from the web and outputs the classification.**

In [ ]:
class ImageClassifier:
    
    def __init__(self):
        self.model = inception_v3.InceptionV3()
    
    def classify(self, img_url):
        response = requests.get(img_url)
        img = PIL.Image.open(io.BytesIO(response.content))
        img = img.resize(inception_dim)
        img_a = keras.preprocessing.image.img_to_array(img)
        img_a = inception_v3.preprocess_input(img_a)
        predictions = self.model.predict(
            numpy.array([img_a])
        )
        predictions_decoded = keras.applications.imagenet_utils.decode_predictions(predictions)[0]
        
        # result
        plt.imshow(img)
        pandas.DataFrame(
            predictions_decoded,
            columns=["category", "label", "value"]
        ).set_index("label").plot(kind="bar", ylim=(0,1))


In [ ]:
img_classifier = ImageClassifier()

In [ ]:
img_classifier.classify("http://s3.amazonaws.com/assets.prod.vetstreet.com/c7/c56260a33911e087a80050568d634f/file/Egyptian-Mau-1-645mk062311.jpg")

In [ ]:
img_classifier.classify("https://i2.cdn.turner.com/money/dam/assets/131011035837-panda-bamboo-1024x576.jpg")

In [ ]:
img_classifier.classify("http://1.bp.blogspot.com/-d7UAmQylLK0/UXGzYtbC9EI/AAAAAAAAADQ/34uJPpP_eCo/s1600/7.jpg")

In [ ]:
img_classifier.classify("https://i.pinimg.com/736x/18/f7/90/18f790b9a2e45f9d7e37174df7714249.jpg")

## References

- [Development of the Inception Network](https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202)

---
_This notebook is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/). Copyright © 2018-2022 [Point 8 GmbH](https://point-8.de)_